<a href="https://colab.research.google.com/github/rootofdata/kostat_AI_contest/blob/main/model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 256->512 ver. CNN-LSTM

In [ ]:
!pip install gensim

In [ ]:
import tensorflow as tf
tf.__version__

'2.8.0'

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [ ]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13232326648598519757
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11320098816
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9238608986752740057
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
xla_global_id: 416903419
]


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
import pandas as pd
import numpy as np
from gensim.models import word2vec
import pickle

# 함수 및 데이터 로드

In [ ]:
def compute_pca(X: np.ndarray, n_components: int=2) -> np.ndarray:
    """Calculate the principal components for X

    Args:
       X: of dimension (m,n) where each row corresponds to a word vector
       n_components: Number of components you want to keep.

    Return:
       X_reduced: data transformed in 2 dims/columns + regenerated original data
    """
    # you need to set axis to 0 or it will calculate the mean of the entire matrix instead of one per row
    X_demeaned = X - X.mean(axis=0)

    # calculate the covariance matrix
    # the default np.cov assumes the rows are variables, not columns so set rowvar to False
    covariance_matrix = np.cov(X_demeaned, rowvar=False)

    # calculate eigenvectors & eigenvalues of the covariance matrix
    eigen_vals, eigen_vecs = np.linalg.eigh(covariance_matrix)

    # sort eigenvalue in increasing order (get the indices from the sort)
    idx_sorted = np.argsort(eigen_vals)

    # reverse the order so that it's from highest to lowest.
    idx_sorted_decreasing = list(reversed(idx_sorted))

    # sort the eigen values by idx_sorted_decreasing
    eigen_vals_sorted = eigen_vals[idx_sorted_decreasing]

    # sort eigenvectors using the idx_sorted_decreasing indices
    # We're only sorting the columns so remember to get all the rows in the slice
    eigen_vecs_sorted = eigen_vecs[:, idx_sorted_decreasing]

    # select the first n eigenvectors (n is desired dimension
    # of rescaled data array, or dims_rescaled_data)
    # once again, make sure to get all the rows and only slice the columns
    eigen_vecs_subset = eigen_vecs_sorted[:, :n_components]

    # transform the data by multiplying the transpose of the eigenvectors 
    # with the transpose of the de-meaned data
    # Then take the transpose of that product.
    X_reduced = np.dot(eigen_vecs_subset.T, X_demeaned.T).T
    return X_reduced

In [ ]:
def get_vector(word):
    if word in word2vec_model:
        return word2vec_model[word]
    else:
        return None

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# train = pd.read_csv('/content/drive/MyDrive/통계청_AI경진대회/final/data/bigword_split_okt2_remove_stopword_8_4')
# test = pd.read_csv('/content/drive/MyDrive/통계청_AI경진대회/final/data/tok_sm8_2_with_remove_stopword')
word2vec_model = word2vec.Word2Vec.load('/content/drive/MyDrive/통계청_AI경진대회/final/data/train_final.model2')
word2vec_model = word2vec_model.wv
submission = pd.read_csv('/content/drive/MyDrive/통계청_AI경진대회/모델 개발용 자료/모델 개발용 자료.csv',header = 1)
data = pd.read_csv('/content/drive/MyDrive/통계청_AI경진대회/실습용 자료/실습용 자료.csv',header = 1)

In [ ]:
train = []
with open('/content/drive/MyDrive/통계청_AI경진대회/final/data/train_final2','rb') as f:
    train = pickle.load(f)
max_len = max(len(l) for l in train)

test = []
with open('/content/drive/MyDrive/통계청_AI경진대회/final/data/submission_final2','rb') as f:
    test = pickle.load(f)
max_len2 = max(len(l) for l in test)

In [ ]:
max_len, max_len2

(53, 49)

# embedding Matrix-> pca

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train)
vocab_size = len(tokenizer.word_index) + 1 # 패딩을 고려하여 +1
print('단어 집합 :',vocab_size)

X_encoded = tokenizer.texts_to_sequences(train)
#print('정수 인코딩 결과 :',X_encoded.shape)

단어 집합 : 37121


In [ ]:
import pickle
with open('/content/drive/MyDrive/통계청_AI경진대회/영석/a_495','rb') as f:
    a_495 = pickle.load(f)

In [ ]:
a1=pd.DataFrame(data['digit_3'])
a2=pd.DataFrame(a_495)
y = pd.concat([a1 ,a2] )
y

,digit_3
0,952
1,472
2,467
3,475
4,872
...,...
490,969
491,970
492,981
493,982


In [ ]:
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

y_train = y.astype(str)

le = LabelEncoder()
le.fit(y_train)
print(le.classes_)
labels = le.classes_

['101' '102' '103' '104' '105' '106' '107' '108' '11' '111' '112' '12'
 '120' '13' '131' '132' '133' '134' '139' '14' '141' '142' '143' '144'
 '15' '151' '152' '161' '162' '163' '171' '172' '179' '181' '182' '191'
 '192' '20' '201' '202' '203' '204' '205' '211' '212' '213' '221' '222'
 '231' '232' '233' '239' '241' '242' '243' '251' '252' '259' '261' '262'
 '263' '264' '265' '266' '271' '272' '273' '274' '281' '282' '283' '284'
 '285' '289' '291' '292' '301' '302' '303' '304' '31' '311' '312' '313'
 '319' '32' '320' '331' '332' '333' '334' '339' '340' '351' '352' '353'
 '360' '370' '381' '382' '383' '390' '411' '412' '421' '422' '423' '424'
 '425' '426' '451' '452' '453' '461' '462' '463' '464' '465' '466' '467'
 '468' '471' '472' '473' '474' '475' '476' '477' '478' '479' '491' '492'
 '493' '494' '495' '501' '502' '51' '511' '512' '52' '521' '529' '551'
 '559' '561' '562' '581' '582' '591' '592' '601' '602' '61' '611' '612'
 '62' '620' '631' '639' '641' '642' '649' '651' '652' '653' '6

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
print(labels)
y_train = le.transform(y_train)
y_train = tf.keras.utils.to_categorical(y_train)
print(y_train.shape) 

['101' '102' '103' '104' '105' '106' '107' '108' '11' '111' '112' '12'
 '120' '13' '131' '132' '133' '134' '139' '14' '141' '142' '143' '144'
 '15' '151' '152' '161' '162' '163' '171' '172' '179' '181' '182' '191'
 '192' '20' '201' '202' '203' '204' '205' '211' '212' '213' '221' '222'
 '231' '232' '233' '239' '241' '242' '243' '251' '252' '259' '261' '262'
 '263' '264' '265' '266' '271' '272' '273' '274' '281' '282' '283' '284'
 '285' '289' '291' '292' '301' '302' '303' '304' '31' '311' '312' '313'
 '319' '32' '320' '331' '332' '333' '334' '339' '340' '351' '352' '353'
 '360' '370' '381' '382' '383' '390' '411' '412' '421' '422' '423' '424'
 '425' '426' '451' '452' '453' '461' '462' '463' '464' '465' '466' '467'
 '468' '471' '472' '473' '474' '475' '476' '477' '478' '479' '491' '492'
 '493' '494' '495' '501' '502' '51' '511' '512' '52' '521' '529' '551'
 '559' '561' '562' '581' '582' '591' '592' '601' '602' '61' '611' '612'
 '62' '620' '631' '639' '641' '642' '649' '651' '652' '653' '6

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(1000495, 232)


In [ ]:
X_train = pad_sequences(X_encoded, maxlen=max_len, padding='post')
print('패딩 결과 :')
print(X_train)
print('y_train numclass',y_train.shape)

패딩 결과 :
[[  92   18   87 ...    0    0    0]
 [1141  183  147 ...    0    0    0]
 [  38  462  331 ...    0    0    0]
 ...
 [3810  952 2807 ...    0    0    0]
 [3810  952 2807 ...    0    0    0]
 [1172 1220  368 ...    0    0    0]]
y_train numclass (1000495, 232)


In [ ]:
numclass = data['digit_3'].nunique()

embedding_matrix = np.zeros((vocab_size, 100))
print('임베딩 행렬의 크기(shape) :',np.shape(embedding_matrix))

임베딩 행렬의 크기(shape) : (37121, 100)


In [ ]:
for word, index in tokenizer.word_index.items():
    # 단어와 맵핑되는 사전 훈련된 임베딩 벡터값
    vector_value = get_vector(word)
    if vector_value is not None:
        embedding_matrix[index] = vector_value

## PCA 100 -> 50

In [ ]:
X = embedding_matrix
X_reduced = compute_pca(X, n_components=50)

In [ ]:
y_train = y_train.astype(int)

In [ ]:
X_reduced.shape

(37121, 50)

# fine tuning

In [ ]:
!pip install optuna

     |████████████████████████████████| 308 kB 4.1 MB/s 
     |████████████████████████████████| 81 kB 9.8 MB/s 
     |████████████████████████████████| 210 kB 48.3 MB/s 
     |████████████████████████████████| 78 kB 6.5 MB/s 
     |████████████████████████████████| 150 kB 50.6 MB/s 
     |████████████████████████████████| 49 kB 5.9 MB/s 
     |████████████████████████████████| 113 kB 40.2 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=0f9d03fea96b2ea85b259ebc2746efa4eb950b652852ac63d3c79f0ea24d4e95
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Activation, Dropout, LSTM, Dense,Conv1D,MaxPooling1D,Embedding,Flatten
from tensorflow.keras import layers
from tensorflow.keras import utils
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf


def build_cnn_lstm2(recurrent_dropout = 0.5,dropout=0.5,):
    with tf.device('/gpu:0'):
        model = Sequential()

        model.add(Input(shape=(max_len,)))
        e = Embedding(vocab_size,output_dim =50,  weights=[X_reduced], input_length=max_len, trainable=False)
        model.add(e)

        model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Dropout(dropout))

        model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
        model.add(MaxPooling1D(pool_size=2))

        model.add(LSTM(256, recurrent_dropout=recurrent_dropout,activation='tanh',kernel_initializer="he_normal",unroll=True,return_sequences=True))
        model.add(Dropout(dropout))

        model.add(LSTM(512, recurrent_dropout=recurrent_dropout,activation='tanh',kernel_initializer="he_normal",unroll=True))
        model.add(Dropout(dropout))

        model.add(Flatten())
        model.add(Dense(y_train.shape[1], activation='softmax'))
        # adam = tf.keras.optimizers.Adam(learning_rate = 1e-5)
        

        return model

In [ ]:
import optuna
from optuna.trial import TrialState
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
def objective(trial):
    x_tri, X_val, y_tri, y_val = train_test_split(X_train,y_train, test_size=0.2, shuffle=False)
    params = {
    # 'optimizer_name' : trial.suggest_categorical("optimizer", ["Adam","RMSprop"]),
    # 'epochs' : trial.suggest_int("epochs", 30, 60,step=10),
    # 'batchsize' : trial.suggest_int("batchsize", max_len, 8*max_len,step=max_len),
    'learning_rate' : trial.suggest_uniform('learning_rate',0.00001,0.0001),
    'recurrent_dropout' : trial.suggest_float('recurrent_dropout',0.1,0.7,step=0.1),
    'dropout' : trial.suggest_float('dropout',0.1,0.7,step=0.1)
    }
    model = build_cnn_lstm2(params['recurrent_dropout'],params['dropout'])
    
    optimizer = tf.keras.optimizers.Adam(learning_rate = params['learning_rate'])

    
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    callbacks = [EarlyStopping(monitor='val_loss', patience=10)]
    history = model.fit(x_tri,y_tri,
                epochs=40,callbacks=callbacks,
              batch_size = 106, validation_split = 0.2,verbose=1)
    
    val_acc = model.evaluate(X_val,y_val)[1]
    weights = model.get_weights()
    # Handle pruning based on the intermediate value.
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

    trial.set_user_attr(key="best_model_weights", value=weights)
    return val_acc

def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key="best_model_weights", 
                            value=trial.user_attrs["best_model_weights"])

study = optuna.create_study(direction="maximize",sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=20, timeout=None, callbacks=[callback])

[I 2022-04-10 15:15:14,383] A new study created in memory with name: no-name-76ca2a6e-7436-4712-a1a0-3d2c454db498


Epoch 1/40
3905/6041 [==================>...........] - ETA: 2:25 - loss: 2.7532 - accuracy: 0.3828

In [ ]:
best_params = study.best_trial.params
print(f"Best trial :{study.best_trial.value}")

Best trial :0.8759114146232605


In [ ]:
best_params

{'optimizer': 'Adam',
 'epochs': 40,
 'batchsize': 53,
 'learning_rate': 8.382620250945622e-05}

In [ ]:
model = build_cnn_lstm2()

if best_params['optimizer'] == "Adam":
    optimizer = tf.keras.optimizers.Adam(learning_rate = best_params['learning_rate'])
else :
    optimizer = tf.keras.optimizers.RMSprop(learning_rate = best_params['learning_rate'])

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
callbacks = [EarlyStopping(monitor='val_loss', patience=10)]
history = model.fit(X_train,y_train, epochs=best_params['epochs'],callbacks=callbacks,batch_size = best_params['batchsize'],recurrent_dropout=best_params['recurrent_dropout'], validation_split = 0.2,verbose=1)

Epoch 1/40
 3015/15102 [====>.........................] - ETA: 11:46 - loss: 2.0122 - accuracy: 0.5416

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show();

# test padding

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(test)
X_encoded = tokenizer.texts_to_sequences(test)
#print('정수 인코딩 결과 :',X_encoded.shape)

In [ ]:
X_test = pad_sequences(X_encoded, maxlen=max_len, padding='post')

In [ ]:
y_pred = model.predict(X_test)
predicted = y_pred.argmax(axis=-1)

In [ ]:
digit_1_index = [1.0, 5.0, 10.0, 35.0, 36.0, 41.0, 45.0, 49.0, 55.0, 58.0, 64.0, 68.0, 70.0, 74.0, 84.0, 85.0, 86.0, 90.0, 94.0, 97.0, 99.0]
digit_1_value = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M','N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U']

dsa = [ i for i in range(100)]
def search_digit_1(i):
    ans = 0
    if i <=digit_1_index[0]:
        ans = digit_1_value[0]
    elif i <=digit_1_index[1]:
        ans=digit_1_value[1]
    elif i <=digit_1_index[2]:
        ans=digit_1_value[2]

    elif i <=digit_1_index[3]:
        ans=digit_1_value[3]
    elif i <=digit_1_index[4]:
        ans=digit_1_value[4]

    elif i <=digit_1_index[5]:
        ans=digit_1_value[5]
    elif i <=digit_1_index[6]:
        ans=digit_1_value[6]

    elif i <=digit_1_index[7]:
        ans=digit_1_value[7]
    elif i <=digit_1_index[8]:
        ans=digit_1_value[8]
    elif i <=digit_1_index[9]:
        ans=digit_1_value[9]
    elif i <=digit_1_index[10]:
        ans=digit_1_value[10]
    elif i <=digit_1_index[11]:
        ans=digit_1_value[11]

    elif i <=digit_1_index[12]:
        ans=digit_1_value[12]
    elif i <=digit_1_index[13]:
        ans=digit_1_value[13]

    elif i <=digit_1_index[14]:
        ans=digit_1_value[14]
    elif i <=digit_1_index[15]:
        ans=digit_1_value[15]

    elif i <=digit_1_index[16]:
        ans=digit_1_value[16]
    elif i <=digit_1_index[17]:
        ans=digit_1_value[17]

    elif i <=digit_1_index[18]:
        ans=digit_1_value[18]
    elif i <=digit_1_index[19]:
        ans=digit_1_value[19]

    elif i <=digit_1_index[20]:
        ans=digit_1_value[20]
    return ans

In [ ]:
for i in range(len(predicted)):
    if len(i) == 2:
        submission['digit_3'][i] = int(labels[predicted[i]])
        submission['digit_2'][i] = int(labels[predicted[i]][0])
        submission['digit_1'][i] = search_digit_1(int(labels[predicted[i]][0]))
    submission['digit_3'][i] = int(labels[predicted[i]])
    submission['digit_2'][i] = int(labels[predicted[i]][:2])
    submission['digit_1'][i] = search_digit_1(int(labels[predicted[i]][0])
submission

In [ ]:
submission[['digit_2','digit_3']] = submission[['digit_2','digit_3']].astype(int)

In [ ]:
submission.to_csv('C:/Users/SEO/OneDrive - 숭실대학교 - Soongsil University/바탕 화면/통계청 ai 경진대회/final/data/sub1_optuna.csv', index=False)

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
